In [ ]:
from glob import glob
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # Hide TF deprecation messages
import random
import numpy as np
import pickle
import modules
import data_utils

In [ ]:
BATCH_SIZE=16

In [ ]:
chair_meshes_paths = list(glob("chairs_ngon/*.obj"))

In [ ]:
with open("chairs_split_dict.pickle", 'rb') as f:
    chairs_split_dict = pickle.load(f)

In [ ]:
chairs_train = []
chairs_val = []
chairs_test = []
for c in chair_meshes_paths: 
    try:
        split = chairs_split_dict[c.split("/")[-1].replace(".obj", "")]
    except KeyError:
#         print(c.split("/")[-1].replace(".obj", ""))
        continue
    if split =='train':
        chairs_train.append(c)
    elif split =='val':
        chairs_val.append(c)
    else:
        chairs_test.append(c)

In [ ]:
train_paths = chairs_train.copy()
random.shuffle(train_paths)

In [ ]:
val_paths = chairs_val.copy()
random.shuffle(val_paths)

In [ ]:
with open('chair_to_text.pickle', 'rb') as f:
    chair_to_text = pickle.load(f)

In [ ]:
with open('word_to_int.pickle', 'rb') as f:
    word_to_int = pickle.load(f)

In [ ]:
with open('int_to_word.pickle', 'rb') as f:
    int_to_word = pickle.load(f)

In [ ]:
lengths = []
count = 0
for key, val in chair_to_text.items():
    for t in val:
        lengths.append(len(t))
        count+=1
max_length = np.array(lengths)
(max_length < 30).sum()/count

In [ ]:
max_length = 30

In [ ]:
def text2shape(paths):
    for path in paths:
        mesh_dict = data_utils.load_process_mesh(path)
        if len(mesh_dict['vertices'])>800:
            continue
        if len(mesh_dict['faces'])>2800:
            continue
        try:
            texts =  chair_to_text[path.split("/")[-1].replace(".obj", "")]
        except KeyError:
            continue
        text = np.random.choice(texts)[:max_length]
        mesh_dict['text_feature'] = np.pad(text, (0,max_length-len(text)))
        yield mesh_dict

In [ ]:
Text2ShapeDataset = tf.data.Dataset.from_generator(
        lambda:text2shape(train_paths),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
            'text_feature':tf.TensorShape(max_length)})

In [ ]:
vertex_model_dataset = data_utils.make_vertex_model_dataset(Text2ShapeDataset, apply_random_shift=True)
vertex_model_dataset = vertex_model_dataset.repeat()
vertex_model_dataset = vertex_model_dataset.padded_batch(BATCH_SIZE, padded_shapes=vertex_model_dataset.output_shapes)
vertex_model_dataset = vertex_model_dataset.prefetch(1)
it = vertex_model_dataset.make_initializable_iterator()
vertex_model_batch = it.get_next()
iterator_init_op_train = it.initializer

In [ ]:
Text2ShapeDatasetVal = tf.data.Dataset.from_generator(
        lambda:text2shape(val_paths),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
            'text_feature':tf.TensorShape(max_length)})
vertex_model_dataset_val = data_utils.make_vertex_model_dataset(Text2ShapeDatasetVal, apply_random_shift=False)
vertex_model_dataset_val = vertex_model_dataset_val.repeat()
vertex_model_dataset_val = vertex_model_dataset_val.padded_batch(BATCH_SIZE, padded_shapes=vertex_model_dataset_val.output_shapes)
vertex_model_dataset_val = vertex_model_dataset_val.prefetch(1)
itv = vertex_model_dataset_val.make_initializable_iterator()
vertex_model_batch_val = itv.get_next()
iterator_init_op_val = itv.initializer

In [ ]:
o = (i for i in text2shape(train_paths))
TRAIN_SIZE = sum(1 for _ in o)
o = (i for i in text2shape(val_paths))
VAL_SIZE = sum(1 for _ in o)

In [ ]:
TRAIN_SIZE

In [ ]:
VAL_SIZE

In [ ]:
vertex_model = modules.TextToVertexModel(
    decoder_config=dict(
      hidden_size=128,
      fc_size=512,
      num_heads=8,
      layer_norm=True,
      num_layers=24,
      dropout_rate=0.4,
      re_zero=True,
      memory_efficient=True
      ),
    path_to_embeddings="glove/glove.6B.100d.txt",
    embedding_dims = 100,
    quantization_bits=8,
    vocab=word_to_int,
    max_num_input_verts=800,  # number of vertices in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    use_discrete_embeddings=True
)

In [ ]:
vertex_model_pred_dist = vertex_model(vertex_model_batch, is_training=True)
vertex_model_loss = -tf.reduce_sum(
    vertex_model_pred_dist.log_prob(vertex_model_batch['vertices_flat']) *
    vertex_model_batch['vertices_flat_mask'])
vertex_samples = vertex_model.sample(
    BATCH_SIZE, context=vertex_model_batch, max_sample_length=800, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

In [ ]:
vertex_model_pred_dist_val = vertex_model(vertex_model_batch_val)
vertex_model_loss_val = -tf.reduce_sum(
    vertex_model_pred_dist_val.log_prob(vertex_model_batch_val['vertices_flat']) *
    vertex_model_batch_val['vertices_flat_mask'])
vertex_samples_val = vertex_model.sample(
    BATCH_SIZE, context=vertex_model_batch_val, max_sample_length=800, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

In [ ]:
face_module_config=dict(
  encoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=10,
      dropout_rate=0.2,
      re_zero=True,
      memory_efficient=True,
      ),
  decoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=14,
      dropout_rate=0.2,
      re_zero=True,
      memory_efficient=True,
      ),
  class_conditional=False,
  decoder_cross_attention=True,
  use_discrete_vertex_embeddings=True,
  max_seq_length=8000,
  )
face_model=modules.FaceModel(**face_module_config)

In [ ]:
face_samples_val = face_model.sample(
    context=vertex_samples_val, max_sample_length=2800, top_p=0.95,
    only_return_complete=False)

In [ ]:
face_model_saver = tf.train.Saver(var_list=face_model.variables)

In [ ]:
#borrowed from https://github.com/optas/shapeglot
def token_ints_to_sentence(tokens, int_to_word):
    text = [int_to_word[i] for i in tokens]
    text = ' '.join(text)
    stop = text.find('<EOS>')
    if stop == -1:
        stop = len(text)
    text = text[:stop]
    return text

In [ ]:
%matplotlib inline
from tqdm import trange
import os

last_saver_vertex = tf.train.Saver(var_list=vertex_model.variables) # will keep last 5 epochs
best_saver_vertex = tf.train.Saver(var_list=vertex_model.variables, max_to_keep=2)  # only keep 1 best

learning_rate = 3e-4
training_steps = 500
check_step_metrics = 50
check_step_samples = 50
EPOCHS = 500
optimizer = tf.train.AdamOptimizer(learning_rate)
vertex_model_optim_op = optimizer.minimize(vertex_model_loss, var_list=vertex_model.variables)
best_v_loss = float('inf')
# Training loop
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    face_model_saver.restore(sess, "face_model/model")

    for e in range(EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        num_steps = (TRAIN_SIZE + BATCH_SIZE - 1) // BATCH_SIZE
        sess.run(iterator_init_op_train)
        t = trange(num_steps)
        loss_values = []
        for i in t:
            sess.run(vertex_model_optim_op)

            loss_val = sess.run(vertex_model_loss)
            loss_values.append(loss_val)
 
            # Log the loss in the tqdm progress bar
            t.set_postfix(loss='{:05.3f}'.format(loss_val))
        mean_loss = np.array(loss_values).mean()
        print("- Train loss vertex: " + str(mean_loss))
        
        last_save_path = os.path.join("text_vertex_shapeglot", 'last')
        last_saver_vertex.save(sess, last_save_path, global_step=e + 1)
    
        num_steps = (VAL_SIZE + BATCH_SIZE - 1) // BATCH_SIZE
        sess.run(iterator_init_op_val)
        loss_values = []
        t = trange(num_steps)
        for i in t:
            loss_val = sess.run(vertex_model_loss_val)
            loss_values.append(loss_val)
            t.set_postfix(loss='{:05.3f}'.format(loss_val))
        mean_loss = np.array(loss_values).mean()
        print("- Eval loss vertex: " + str(mean_loss))
 
        if mean_loss<=best_v_loss:
            best_v_loss = mean_loss
            best_save_path = os.path.join("text_vertex_shapeglot", 'best')
            best_save_path = best_saver_vertex.save(sess, best_save_path, global_step=e + 1)
            print("- Found new best vertex model, saving in {}".format(best_save_path))

        if e>150 and e % check_step_samples==0:
            sess.run(iterator_init_op_val)
            v_samples_np, f_samples_np, b_np = sess.run((vertex_samples_val, face_samples_val, vertex_model_batch_val))
            mesh_list = []
            for n in range(BATCH_SIZE):
                print(token_ints_to_sentence(b_np['text_feature'][n], int_to_word))
                mesh_list.append({
                    'vertices': v_samples_np['vertices'][n][:v_samples_np['num_vertices'][n]],
                    'faces': data_utils.unflatten_faces(
                        f_samples_np['faces'][n][:f_samples_np['num_face_indices'][n]])
                    })
            data_utils.plot_meshes(mesh_list, ax_lims=0.5)
